In [12]:
from time import time

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

from scipy import optimize
from sklearn.metrics import accuracy_score
from sklearn.model_selection import KFold
from sklearn.tree import DecisionTreeClassifier

%matplotlib inline

In [13]:
def choice_sub(y, size = 1):
    
    choice = np.zeros(y.shape[0],dtype = bool)
    i = 0
    while (choice[choice == True].shape[0]<size):
        choice[i] = np.random.randint(low = 0,high = 2,size = 1,dtype = bool)
        if i == y.shape[0]-1:
            i=0
        else:
            i += 1
    return y[choice]

In [14]:
class MyDecisionTreeClassifier:
    NON_LEAF_TYPE = 0
    LEAF_TYPE = 1

    def __init__(self, min_samples_split=2, max_depth=float('+inf'), sufficient_share=1.0, criterion='gini',\
                 max_features=None):
        self.tree = dict()
        self.min_samples_split = min_samples_split
        self.max_depth = max_depth
        self.sufficient_share = sufficient_share
        self.num_class = -1
        if criterion == 'gini':
            self.G_function = self.__gini
        elif criterion == 'entropy':
            self.G_function = self.__entropy
        elif criterion == 'misclass':
            self.G_function = self.__misclass
        else:
            print 'invalid criterion name'
            raise

        if max_features == 'sqrt':
            self.get_feature_ids = self.__get_feature_ids_sqrt
        elif max_features == 'log2':
            self.get_feature_ids = self.__get_feature_ids_log2
        elif max_features == None:
            self.get_feature_ids = self.__get_feature_ids_N
        else:
            print 'invalid max_features name'
            raise

    def __gini(self, l_c, l_s, r_c, r_s):
        l_s = l_s.astype('float')
        r_s = r_s.astype('float')

        l_c = l_c/l_s;
        r_c = r_c/r_s; 
        s = l_s[0][0]+r_s[0][0]
        
        l_s = l_s.reshape(l_s.shape[0],)
        r_s = r_s.reshape(r_s.shape[0],)
        
        return (l_s*(l_c*l_c).sum(axis = 1)+r_s*(r_c*r_c).sum(axis = 1))/s
    
    def __entropy(self, l_c, l_s, r_c, r_s):
        l_s = l_s.astype('float')
        r_s = r_s.astype('float')
        
        l_c = l_c/l_s;
        r_c = r_c/r_s;
        -l_c*np.log2(l_c).sum(axis = 1)
        l_c = np.dot(l_c,np.log(l_c).T)
        r_c = np.dot(r_c,np.log(r_c).T)
        
        s = l_s[0][0]+r_s[0][0]
        
        return (l_s*((-l_c*np.log2(l_c)).sum(axis = 1))+r_s*((-r_c*np.log2(r_c)).sum(axis = 1)))/s

    def __misclass(self, l_c, l_s, r_c, r_s):
        l_s = l_s.astype('float')
        r_s = r_s.astype('float')
        
        l_c = l_c/l_s;
        r_c = r_c/r_s;
        
        s = l_s[0][0]+r_s[0][0]
        
        l_s = l_s/s
        r_s = r_s/s
        
        return (l_s*(1 - np.amax(l_c, axis = 1))+r_s*(1 - np.amax(r_c, axis = 1)))

    def __get_feature_ids_sqrt(self, n_feature):
        feature_ids = range(n_feature)
        np.random.shuffle(feature_ids)
        
        return choise_sub(features_id, int(np.sqrt(n_feature)))
        
    def __get_feature_ids_log2(self, n_feature):
        feature_ids = range(n_feature)
        np.random.shuffle(feature_ids)
        return choise_sub(features_id, int(np.log2(n_feature)))

    def __get_feature_ids_N(self, n_feature):
        
        return np.arange(n_feature, dtype = int)
    
    def __sort_samples(self, x, y):
        sorted_idx = x.argsort()
        return x[sorted_idx], y[sorted_idx]

    def __div_samples(self, x, y, feature_id, threshold):
        left_mask = x[:, int(feature_id)] > threshold
        right_mask = ~left_mask
        return x[left_mask], x[right_mask], y[left_mask], y[right_mask]

    def __find_threshold(self, x, y):
        #Сортирует точки по столбку признаку х. Подсчитывает кол-во уникальных классов
        sorted_x, sorted_y = self.__sort_samples(x, y)
        class_number = np.unique(y).shape[0]
        
        # Отрезание по краям массива хвостов длинной self.min_samples_split.
        #Затем проверка оставшейся части на наличие изменений. Если все оставшиеся элементы одинаковы,
        #то возвращаем бесконечность и None, если же разные, то возвращаем массив,который показывает, сколько
        #точек слева при каждом переходе от одного индекса,где произошло изменение класса, к другому
        splitted_sorted_y = sorted_y[self.min_samples_split:-self.min_samples_split]
        r_border_ids = np.where(splitted_sorted_y[:-1] != splitted_sorted_y[1:])[0] + (self.min_samples_split + 1)
        
        if len(r_border_ids) == 0:
            return float('+inf'), None
        
        #Этот массив, который показывает, так сказать, длительность постоянства класса: [1,1,2,1,3] будет означать, что
        #сначала был один класс, потом он сменился сразу, потом еще раз произошла смена сразу, затем смена произошла через
        #один, смена сразу, смена через два. То есть массив r_border_ids выглядил, например так: [0,1,2,2,3,4,4,4]
        eq_el_count = r_border_ids - np.append([self.min_samples_split], r_border_ids[:-1]) 
        # Создается матрица размером кол-во переходов в splitted_sorted_y * кол-во классов
        one_hot_code = np.zeros((r_border_ids.shape[0], class_number))
        # Если i-ый переход был сделан из класса j, то в one_hot_code[i][j] ставится 1 
        one_hot_code[np.arange(r_border_ids.shape[0]), sorted_y[r_border_ids - 1]] = 1
        # Каждая строчка умножается на длительность класса, из которого был сделан переход
        class_increments = one_hot_code * eq_el_count.reshape(-1, 1)
        # Это очень странно, но тут прибавляют к первой строке, строку обозначающую кол-во входов каждого класса
        #в начальный хвост !неотсортированного! массива y!(хотя до этого работали с отсорированным), поэтому я тут исправил
        #сортированный....
        class_increments[0] = class_increments[0] + np.bincount(sorted_y[:self.min_samples_split], minlength=class_number)
        
        # Матрица, где i это по сути ползунок, которым мы отсеиваем элементы в левое поддерево, а j это кол-во точек
        #с данным классом в левом поддереве
        l_class_count = np.cumsum(class_increments, axis=0)
        # Тоже самое только для правого дерева
        r_class_count = np.bincount(y) - l_class_count
        # Транспоинровали r_border_ids... столбец, обозначающий кол-во точек в левом поддереве при каждой позиции i ползунка 
        l_sizes = r_border_ids.reshape(l_class_count.shape[0], 1)
        # Тоже самое для правого
        r_sizes = sorted_y.shape[0] - l_sizes
        # Для каждого разбиения находит меру неопределенности, и выбирает разбиение при котором
        #достигается минимум
        gs = self.G_function(l_class_count, l_sizes, r_class_count, r_sizes)
        idx = np.argmin(gs)
    
        #узнается позиция, на которой произошел переход в нашем изначальном массиве и возвращаются две величины:
        #значение меры неопределенности на выбранном разбиении и значение, которое будет использоваться в условии,
        #которое будет разбивать на два поддерева
        left_el_id = l_sizes[idx][0]
        return gs[idx], (sorted_x[left_el_id-1] + sorted_x[left_el_id]) / 2.0

    def __fit_node(self, x, y, node_id, depth, pred_f=-1):
        # Ваш код
        # Необходимо использовать следующее
        if depth < self.max_depth:
            if np.unique(y).shape[0]!=1:
                index = self.get_feature_ids(x.shape[1])
                v = np.empty((index.shape[0],2))
                for i in xrange(len(index)):
                    v[i][0], v[i][1] = self.__find_threshold(x[:,index[i]],y)
                min_ = v[:,0].argmin()
                if v[min_][0] != float("+inf"):
                    self.tree[node_id] = np.array([0,index[min_],v[min_][1]])
                else:
                    b = np.bincount(y, minlength = np.unique(y).shape[0])
                    self.tree[node_id] = np.array([1,b.argmax(),None])
                    return
                x_left, x_right,y_left, y_right = self.__div_samples(x,y,self.tree[node_id][1],self.tree[node_id][2])
                if y_left.shape[0] != 0 and y_right.shape[0] != 0:
                    self.__fit_node(x_left, y_left, 2*node_id + 1, depth + 1)
                    self.__fit_node(x_right, y_right, 2*node_id + 2, depth + 1)
                else:
                    self.tree[node_id][0]=1
                    b = np.bincount(y, minlength = np.unique(y).shape[0])
                    self.tree[node_id][1] = b.argmax()
                    return
            else:    
                self.tree[node_id] = np.array([1,y[0],None])
                return
        else:
            b = np.bincount(y, minlength = np.unique(y).shape[0])
            self.tree[node_id] = np.array([1,b.argmax(),None])
            return
        pass
    
    def fit(self, x, y):
        self.num_class = np.unique(y).size
        self.__fit_node(x, y, 0, 0) 

    def __predict_class(self, x, node_id):
        node = self.tree[node_id]
        if node[0] == self.__class__.NON_LEAF_TYPE:
            _, feature_id, threshold = node
            if x[int(feature_id)] > threshold:
                return self.__predict_class(x, 2 * node_id + 1)
            else:
                return self.__predict_class(x, 2 * node_id + 2)
        else:
            return node[1]

    def __predict_probs(self, x, node_id):
        node = self.tree[node_id]
        if node[0] == self.__class__.NON_LEAF_TYPE:
            _, feature_id, threshold = node
            if x[int(feature_id)] > threshold:
                return self.__predict_probs(x, 2 * node_id + 1)
            else:
                return self.__predict_probs(x, 2 * node_id + 2)
        else:
            return node[2]
        
    def predict(self, X):
        return np.array([self.__predict_class(x, 0) for x in X])
    
    def predict_probs(self, X):
        return np.array([self.__predict_probs(x, 0) for x in X])

    def fit_predict(self, x_train, y_train, predicted_x):
        self.fit(x_train, y_train)
        return self.predict(predicted_x)

In [15]:
df = pd.read_csv('./cs-training.csv', sep=',').dropna()
df.head()

,SeriousDlqin2yrs,RevolvingUtilizationOfUnsecuredLines,age,NumberOfTime30-59DaysPastDueNotWorse,DebtRatio,MonthlyIncome,NumberOfOpenCreditLinesAndLoans,NumberOfTimes90DaysLate,NumberRealEstateLoansOrLines,NumberOfTime60-89DaysPastDueNotWorse,NumberOfDependents
1,1,0.766127,45,2,0.802982,9120.0,13,0,6,0,2.0
2,0,0.957151,40,0,0.121876,2600.0,4,0,0,0,1.0
3,0,0.658180,38,1,0.085113,3042.0,2,1,0,0,0.0
4,0,0.233810,30,0,0.036050,3300.0,5,0,0,0,0.0
5,0,0.907239,49,1,0.024926,63588.0,7,0,1,0,0.0


In [16]:
x = df.as_matrix(columns=df.columns[1:])
y = df.as_matrix(columns=df.columns[:1])
y = y.reshape(y.shape[0])
print y.shape

(120269,)


In [17]:
my_clf = MyDecisionTreeClassifier(min_samples_split=2)
clf = DecisionTreeClassifier(min_samples_split=2)

## Проверка скорости работы

In [18]:
t1 = time()
my_clf.fit(x, y)
t2 = time()
print(t2 - t1)

t1 = time()
clf.fit(x, y)
t2 = time()
print(t2 - t1)

1.09039211273
0.829087018967


In [19]:
len(my_clf.tree)

723

## Проверка качества работы

In [20]:
gkf = KFold(n_splits=5, shuffle=True)

In [21]:
for train, test in gkf.split(x, y):
    X_train, y_train = x[train], y[train]
    X_test, y_test = x[test], y[test]
    my_clf.fit(X_train, y_train)
    a = my_clf.predict(X_test)
    print(accuracy_score(y_pred=a, y_true=y_test))
    #print np.where(a==1)

0.929658268895
0.932443668413
0.929117818242
0.931071755217
0.930902590113


In [22]:
for train, test in gkf.split(x, y):
    X_train, y_train = x[train], y[train]
    X_test, y_test = x[test], y[test]
    clf.fit(X_train, y_train)
    print(accuracy_score(y_pred=clf.predict(X_test), y_true=y_test))


0.889249189324
0.890912114409
0.888999750561
0.893863806436
0.889951357419
